In [ ]:
#imports packages

import keras
import tensorflow.keras.optimizers
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from google.colab import drive
from nilearn import datasets
from nilearn.connectome import ConnectivityMeasure
from nilearn import image
from nilearn.regions import RegionExtractor
from nilearn import plotting
import matplotlib.pyplot as plt
from nilearn import decomposition
import pandas as pd
import os
from nilearn.image import mean_img, index_img
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
!pip install nilearn

In [ ]:
#mounts the google drive to read from it

drive.mount('/content/drive')

In [ ]:
#reads the adhd_200 data sample from nilearn

num = 40
adhd_data = datasets.fetch_adhd(n_subjects=num) #reads the fMRI dataset

In [ ]:
#extracts the phenotypic data from the sample dataset

dataa = adhd_data.phenotypic
dataa = pd.DataFrame(dataa)

In [ ]:
#gets the ohsu data from the drive

DIR = "drive/MyDrive/fmri_ohsu"

ohsu = [DIR + "/" + entry for entry in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, entry))]
ohsu = ohsu[1:]

ohsuu = []
for i in range(len(ohsu)):
  if ohsu[i].split("_")[-1].split(".")[0] == 'run1':
    ohsuu.append(ohsu[i])

print(len(ohsuu))

In [ ]:
#gets the neuroimage data from the drive

DIR = "drive/MyDrive/fmri_neuroimage"

neuroimage = [DIR + "/" + entry for entry in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, entry))]
print(len(neuroimage))

In [ ]:
#gets the pittsburgh data from the drive

DIR = "drive/MyDrive/fmri-pittsburgh"

pittsburgh = [DIR + "/" + entry for entry in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, entry))]
print(len(pittsburgh))

In [ ]:
#gets the peking data from the drive

DIR = "drive/MyDrive/fmri_peking"

peking = [DIR + "/" + entry for entry in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, entry))]
print(len(peking))

In [ ]:
#combines the multiple sites functional data

func = ohsuu + neuroimage + peking
print(len(func))

In [ ]:
#extracts subject IDs
IDs = []

for i in range(len(func)):
  IDs.append(np.int64(func[i].split("_")[3]))

#sorts the subject IDs to match subjects later
IDs_sort = sorted(IDs)

#stores functional data in a new list with the new IDs order
funcy = []
for elem in IDs_sort:
  for i in range(len(IDs_sort)):
    if np.int64(func[i].split("_")[3]) == elem:
      funcy.append(func[i])

In [ ]:
#stores the IDs from the new list to check matching later
IsDs = []

for i in range(len(funcy)):
  IsDs.append(np.int64(funcy[i].split("_")[3]))

Pheno tsv

In [ ]:
#gets the ohsu phenotypic data from the drive

site_ohsu = int(pheno[pheno["ScanDir ID"] == 1084283]["Site"])

pheno_ohsu = pheno[pheno["Site"] == site_ohsu]

pheno_ohsu = pheno_ohsu.reset_index()

print(len(pheno_ohsu))

In [ ]:
#gets the neuroimage phenotypic data from the drive

site_neuroimage = int(pheno[pheno["ScanDir ID"] == 1017176]["Site"])

pheno_neuroimage = pheno[pheno["Site"] == site_neuroimage]

pheno_neuroimage = pheno_neuroimage.reset_index()

len(pheno_neuroimage)

In [ ]:
#gets the peking phenotypic data from the drive

site_peking = int(pheno[pheno["ScanDir ID"] == 1050345]["Site"])

pheno_peking = pheno[pheno["Site"] == site_peking]

pheno_peking = pheno_peking.reset_index()

len(pheno_peking)

In [ ]:
#combines the multiple sites phenotypic data

diag = pd.concat([pheno_ohsu, pheno_neuroimage, pheno_peking], axis = 0)

diag = diag.reset_index()

Matching them

In [ ]:
#matches subject IDs of functional and phenotypic data
df_pheno = pd.DataFrame()

for elem in IDs_sort:
  for i in range(len(diag)):
    if diag.loc[i,"ScanDir ID"] == elem:
      df_pheno = df_pheno.append(diag.iloc[i,:], ignore_index = True)



In [ ]:
IDs_match = []
for i in list(df_pheno["ScanDir ID"]):
  IDs_match.append(int(i))

#checks that IDs match
IDs_match == IsDs

In [ ]:
#gets the phenotypic data of just the peking university data as a subset (ordered)

pekingy = df_pheno[df_pheno["Site"] == 1].index

pekingy = np.array(pekingy)

pekingy_pheno = df_pheno[df_pheno["Site"] == 1]

In [ ]:
#converts to numeric to work with later
df_pheno["Verbal IQ"] = df_pheno["Verbal IQ"].apply(pd.to_numeric, errors='coerce')
df_pheno["Performance IQ"] = df_pheno["Performance IQ"].apply(pd.to_numeric, errors='coerce')
df_pheno["Full4 IQ"] = df_pheno["Full4 IQ"].apply(pd.to_numeric, errors='coerce')
df_pheno["Handedness"] = df_pheno["Handedness"].apply(pd.to_numeric, errors='coerce')

#stores only the performance (visuospatial) IQ for later purposes
phenotypic = df_pheno[["Performance IQ"]]
phenotypic = np.array(phenotypic)

In [ ]:
#gets the diagnosis labels, and merge all 3 ADHD subsets to one (ADHD vs not)

y = np.array(df_pheno["DX"])

for i in range(len(y)):
  if y[i] == "1" or y[i] == "2" or y[i] == "3":
    y[i] = "1"

y = y.astype(int)

print(len(y))

In [ ]:
#gets the diagnosis data of just the peking site data as a subset (ordered)

y_peking = y[np.ix_(list(pekingy))]

print(len(y_peking))

DictLearn

Code below from nilearn documentation https://nilearn.github.io/stable/index.html


In [ ]:
from nilearn.decomposition import DictLearning

# Initialize DictLearning object
dict_learn = DictLearning(n_components=8, smoothing_fwhm=6.,
                          memory="nilearn_cache", memory_level=2,
                          random_state=0)
# Fit to the data
dict_learn.fit(peking)
# Resting state networks/maps in attribute `components_img_`
components_img = dict_learn.components_img_

# Visualization of functional networks

plotting.plot_prob_atlas(components_img, view_type='filled_contours',
                         title='Dictionary Learning maps')

In [ ]:
masker = dict_learn.masker_
data = np.concatenate(masker.transform(peking))
scores = dict_learn._raw_score(data, per_component=True)
plt.plot(scores, '-o')
plt.title('Explained variance for 100 components')

In [ ]:
#Plots the extracted regions in a specific network

plotting.plot_stat_map(image.index_img(components_img, 7), cut_coords = (3, 20, 36))
plotting.show()

In [ ]:
extractor = RegionExtractor(components_img, threshold=0.5,
                            thresholding_strategy='ratio_n_voxels',
                            extractor='local_regions',
                            standardize=True, min_region_size=1350)
extractor.fit()
# Extracted regions are stored in regions_img_
regions_extracted_img = extractor.regions_img_
# Each region index is stored in index_
regions_index = extractor.index_
# Total number of regions extracted
n_regions_extracted = regions_extracted_img.shape[-1]

# Visualization of region extraction results
title = ('%d regions are extracted from %d components.'
         '\nEach separate color of region indicates extracted region'
         % (n_regions_extracted, 8))
plotting.plot_prob_atlas(regions_extracted_img, view_type='filled_contours',
                         title=title)

In [ ]:
correlations = []
# Initializing ConnectivityMeasure object with kind='correlation'
connectome_measure = ConnectivityMeasure(kind='correlation')
for filename in funcy:
    # call transform from RegionExtractor object to extract timeseries signals
    timeseries_each_subject = extractor.transform(filename)
    # call fit_transform from ConnectivityMeasure object
    correlation = connectome_measure.fit_transform([timeseries_each_subject])
    # saving each subject correlation to correlations
    correlations.append(correlation)

# Mean of all correlations
mean_correlations = np.mean(correlations, axis=0).reshape(n_regions_extracted,
                                                          n_regions_extracted)

At this step I visually inspected the outputs and selected the indices below to correspond to my two networks of interest. I am fitting the DictLearn object to a small subset of the data because of the computationally extensive time.

In [ ]:
np.shape(correlations)

corrs = np.array(correlations)

corrs = corrs.reshape(len(func), n_regions_extracted, n_regions_extracted)

DMN = np.array([10, 13, 14, 17, 32])
ATT = np.array([37, 36, 35, 5, 8,  7, 18, 19, 20])
DMN&ATT = np.array([10, 13, 14, 17, 32, 37, 36, 35, 5, 8,  7, 18, 19, 20])

DMN_conn = corrs[np.ix_(list(range(len(func))), here, here)]

ATT_conn = corrs[np.ix_(list(range(len(func))), here2, here2)]

ATT_DMN_conn = corrs[np.ix_(list(range(len(func))), here_2, here_2)]

# Gets the mean DMN-corresponding cells of connectivity matrix
DMN_conn = mean_correlations[np.ix_(here, here)]

ICA

In [ ]:
# Does ICA decomposition
canica = decomposition.CanICA(n_components=20, mask_strategy='background')
canica.fit(peking)

In [ ]:
components = canica.components_

#Using a masker to project into the 3D space
components_img = canica.masker_.inverse_transform(components)

#Plotting all the components
plotting.plot_prob_atlas(components_img, title='All ICA components')
plotting.show()

In [ ]:
#Using a filter to extract the regions time series 
from nilearn import input_data
masker = input_data.NiftiMapsMasker(components_img, smoothing_fwhm=6,
 standardize=False, detrend=True,
 t_r=2.5, low_pass=0.1,
 high_pass=0.01)
#Computing the regions timeseries signals and storing them
subjects = []

for func_file in peking:
 time_series = masker.fit_transform(func_file)
 subjects.append(time_series)

In [ ]:
#Generates correlation matrices as connectivity measures
correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrices_ICA = correlation_measure.fit_transform(subjects)

Data prep




In [ ]:
def matrix(n_subjects=len(peking), rows, columns, data1, data2, data_pheno):
  '''
  Creates the input variables to the classifier below from the different types
  of data avaialble
  '''
  X = np.zeros(shape=(n_subjects,rows,columns))
  X[:,:np.shape(data1)[0],:np.shape(data1)[1]] = data1
  if data2:
    X[:,np.shape(data1)[0]:, np.shape(data1)[1]:] = data2
  mat = []
  for i in range(n_subjects):
    this = X
    matr = list(this[np.triu_indices(rows, k=1)])
    mat.append(matr)
  if data_pheno:
    mat_all = np.concatenate([mat, data_pheno], axis=1)
    return mat_all
  else:
    return mat


In [ ]:
# call examples to the function

predinfo = matrix(n_subjects, 25, 25, correlation_matrices_ICA, DMN_conn, None)

predinfo2 = matrix(n_subjects, 29, 29, correlation_matrices_ICA, ATT_conn, None)

predinfo3 = matrix(n_subjects, 29, 29, DMN_conn, None, phenotypic)


Tests

(I tried different input data for the different predictor data combinations, and then plotted the results as in my paper)

In [ ]:
#cross-validation score of support vector machine
clf = svm.SVC()
scores = cross_val_score(clf, predinfo, y_peking, cv=7)
print(scores)

In [ ]:
#cross-validation score of a sequential neural network

acc = []
for i in range(7):
  from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = train_test_split(predinfo, y_peking, test_size=0.3)

  import keras
  import tensorflow.keras.optimizers
  from tensorflow.keras.optimizers import Adam
  from keras.models import Sequential
  from keras.layers import Dense
  #from keras.optimizers import Adam
  classifier = Sequential()
  #First Hidden Layer
  classifier.add(Dense(32, activation='tanh', kernel_initializer='random_normal', input_shape=np.shape(predinfo)[1:]))
  #Second Hidden Layer
  classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
  #Third Hidden Layer
  classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
  #Output Layer
  classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
  #Compiling the model
  classifier.compile(optimizer = Adam(lr =.0001),loss='binary_crossentropy', metrics =['accuracy'])
  #Fitting the model
  classifier.fit(np.array(X_train),np.array(y_train), batch_size=32, epochs=100)
  eval_model=classifier.evaluate(np.array(X_train), np.array(y_train))
  acc.append(eval_model[1])